In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from category_encoders import *


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-test/test.csv
/kaggle/input/gender-submission/gender_submission.csv
/kaggle/input/titanic-datasets/train.csv
/kaggle/input/titanic-datasets/test.csv
/kaggle/input/titanic-datasets/gender_submission.csv
/kaggle/input/titanic-train/train.csv


In [29]:
train_data = pd.read_csv("/kaggle/input/titanic-train/train.csv")
X =  (train_data[['Pclass', 'Sex', 'Parch']]).dropna() 
y = pd.DataFrame(train_data.iloc[:, 1]).dropna()




# ["Pclass", "Sex", "SibSp", "Parch"]


# train_data
X
# y_train

# come back and drop is null rows


,Pclass,Sex,Parch
0,3,male,0
1,1,female,0
2,3,female,0
3,1,female,0
4,3,male,0
...,...,...,...
886,2,male,0
887,1,female,0
888,3,female,2
889,1,male,0


In [30]:
# for predictions
test_data= pd.read_csv("/kaggle/input/titanic-test/test.csv")
X_test = (test_data[['Pclass', 'Sex', 'Parch']]).dropna()
X_test

,Pclass,Sex,Parch
0,3,male,0
1,3,female,0
2,2,male,0
3,3,male,0
4,3,female,1
...,...,...,...
413,3,male,0
414,1,female,0
415,3,male,0
416,3,male,0


In [23]:
# Create X_train and y_train and validation sets. to train the model 
from sklearn.model_selection import train_test_split

X_train,X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)  # training and validation sets

# X_train
X_val

,Pclass,Sex,Parch
709,3,male,1
439,2,male,0
840,3,male,0
720,2,female,1
39,3,female,0
...,...,...,...
433,3,male,0
773,3,male,0
25,3,female,5
84,2,female,0



Training Phase:

1. Fit the encoder on the training set (X_train and y_train).
2. Transform the training set using the fitted encoder (X_train_encoded).
Validation Phase:
3. Transform the validation set using the same encoder (X_val_encoded).

Test (or Deployment) Phase:
4. Transform the test set using the same encoder (X_test_encoded).



In [24]:
# Categorical encoding (Target Encoder)

# encode categorical features
X_feature_names = list(X.columns)
# Define catboost encoder
target_encoder = TargetEncoder()
# fit on X_train
X_train_encoded = target_encoder.fit_transform(X_train[X_feature_names],y_train)

X_train_encoded

#sample code for y (not applicable here):
# train_cbe = cbe_encoder.fit_transform(df[feature_list], df[feature_list[-1]])

,Pclass,Sex,Parch
331,1,0.186296,0
733,2,0.186296,0
382,3,0.186296,0
704,3,0.186296,0
813,3,0.738776,2
...,...,...,...
106,3,0.738776,0
270,1,0.186296,0
860,3,0.186296,0
435,1,0.738776,2


In [25]:
# Transform training and validation set using the fitted encoder
X_train_encoded = target_encoder.transform(X_train_encoded[X_feature_names]) # transform X_train
X_val_encoded= target_encoder.transform(X_val[X_feature_names]) # transform X_val
X_test_encoded = target_encoder.transform(X_test[X_feature_names]) #transform X_test

X_test_encoded

/opt/conda/lib/python3.10/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)


,Pclass,Sex,Parch
0,3,0.186296,0
1,3,0.738776,0
2,2,0.186296,0
3,3,0.186296,0
4,3,0.738776,1
...,...,...,...
413,3,0.186296,0
414,1,0.738776,0
415,3,0.186296,0
416,3,0.186296,0


In [43]:
# Model development

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

# create a pipeline to deal with na values
clf_pipeline = make_pipeline(SimpleImputer(strategy = "mean"),RandomForestClassifier())
# flatten y to use for RandomForestClassifier
y_train_flattened = y_train.values.ravel()

titanic_model = clf_pipeline.fit(X_train_encoded, y_train_flattened)

y_pred_val = clf_pipeline.predict(X_val_encoded)

#Hyperparameter tuning
# need to add randomforestclassifier due to pipeline
param_grid = {
    'randomforestclassifier__max_features': [2, 3],
    'randomforestclassifier__min_samples_leaf': [3, 4, 5],
    'randomforestclassifier__min_samples_split': [8, 10, 12],
    'randomforestclassifier__n_estimators': [100, 200, 300, 1000],
    'randomforestclassifier__max_depth': [3,6,9]
    
}



#Grid Search
grid_search = GridSearchCV(clf_pipeline,param_grid, cv=5)
grid_search.fit(X_train_encoded, y_train_flattened) #fit 

# Extract best parameter values
best_params = grid_search.best_params_
print(best_params)

# Now train with best hyperparameters
final_model = RandomForestClassifier()
final_titanic_model = final_model.fit(X_train_encoded, y_train_flattened)
final_y_pred_val = final_model.predict(X_val_encoded)

val_accuracy = final_titanic_model.score(X_val_encoded, y_val)
print(f'Validation Accuracy: {val_accuracy}')

{'randomforestclassifier__max_depth': 3, 'randomforestclassifier__max_features': 2, 'randomforestclassifier__min_samples_leaf': 3, 'randomforestclassifier__min_samples_split': 8, 'randomforestclassifier__n_estimators': 100}
Validation Accuracy: 0.7206703910614525


In [ ]:
# Metrics
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score 

cm = confusion_matrix(y_val, final_y_pred_val)
print(f"Confusion Matrix(Validattion Set): {cm}")
acc = accuracy_score(y_val, final_y_pred_val)
print(f"Accuracy Score (Validation Set): {acc}")
cr = classification_report(y_val, final_y_pred_val)
print(f"Classification Report (Validation Set): {cr}")

In [39]:
# Now on X_test
y_pred_test = final_titanic_model.predict(X_test_encoded)
# print(pd.DataFrame({'PassengerId': test_data.PassengerId.dropna(), 'Survivors': y_pred_test}))
y_pred_test

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,